In [71]:
import requests
from bs4 import BeautifulSoup
import csv
import pandas as pd
import os
from textblob import TextBlob

In [122]:
# URL for YTL Power on Yahoo Finance to get the news
BASE_URL = 'https://finance.yahoo.com'
url = "https://finance.yahoo.com/quote/6742.KL"
response = requests.get(url)
url = "https://finance.yahoo.com/quote/NVDA/"
response1 = requests.get(url)

In [123]:
response

<Response [200]>

In [124]:
response1

<Response [200]>

In [125]:
if response.status_code == 200 & response1.status_code == 200:
    # Get the HTML content of the webpage
    html_content = response.text
    print(html_content)
else:
    print("Failed to retrieve HTML content. Status code:", response.status_code)

<!doctype html><html data-color-theme="light" id="atomic" class="NoJs chrome desktop failsafe" lang="en-US"><head prefix="og: https://ogp.me/ns#"><script>window.performance && window.performance.mark && window.performance.mark('PageStart');</script><meta charset="utf-8"><title>YTL Power International Berhad (6742.KL) Stock Price, News, Quote &amp; History - Yahoo Finance</title><meta name="keywords" content="6742.KL, YTL Power International Berhad, 6742.KL stock chart, YTL Power International Berhad stock chart, stock chart, stocks, quotes, finance"><meta http-equiv="x-dns-prefetch-control" content="on"><meta property="twitter:dnt" content="on"><meta property="fb:app_id" content="458584288257241"><meta name="theme-color" content="#400090"><meta name="viewport" content="width=device-width, initial-scale=1"><meta name="description" lang="en-US" content="Find the latest YTL Power International Berhad (6742.KL) stock quote, history, news and other vital information to help you with your st

In [126]:
#get the HTML content of the webpage
doc = BeautifulSoup(html_content, 'html.parser')
doc

<!DOCTYPE html>
<html class="NoJs chrome desktop failsafe" data-color-theme="light" id="atomic" lang="en-US"><head prefix="og: https://ogp.me/ns#"><script>window.performance && window.performance.mark && window.performance.mark('PageStart');</script><meta charset="utf-8"/><title>YTL Power International Berhad (6742.KL) Stock Price, News, Quote &amp; History - Yahoo Finance</title><meta content="6742.KL, YTL Power International Berhad, 6742.KL stock chart, YTL Power International Berhad stock chart, stock chart, stocks, quotes, finance" name="keywords"/><meta content="on" http-equiv="x-dns-prefetch-control"/><meta content="on" property="twitter:dnt"/><meta content="458584288257241" property="fb:app_id"/><meta content="#400090" name="theme-color"/><meta content="width=device-width, initial-scale=1" name="viewport"/><meta content="Find the latest YTL Power International Berhad (6742.KL) stock quote, history, news and other vital information to help you with your stock trading and investin

In [127]:
# Get the news title
doc.find('title')

<title>YTL Power International Berhad (6742.KL) Stock Price, News, Quote &amp; History - Yahoo Finance</title>

In [128]:
div_tags = doc.find_all('div', {'class': "Ov(h) Pend(44px) Pstart(25px)"})

In [129]:
len(div_tags)

10

In [130]:
print(div_tags[1])

<div class="Ov(h) Pend(44px) Pstart(25px)"><div class="C(#959595) Fz(11px) D(ib) Mb(6px)">Simply Wall St.</div><h3 class="Mb(5px)"><a class="js-content-viewer wafer-caas Fw(b) Fz(18px) Lh(23px) LineClamp(2,46px) Fz(17px)--sm1024 Lh(19px)--sm1024 LineClamp(2,38px)--sm1024 mega-item-header-link Td(n) C(#0078ff):h C(#000) LineClamp(2,46px) LineClamp(2,38px)--sm1024 not-isInStreamVideoEnabled" data-uuid="08ccb2b4-6984-314b-844b-ce72216db8e8" data-wf-caas-prefetch="1" data-wf-caas-uuid="08ccb2b4-6984-314b-844b-ce72216db8e8" href="/news/eps-growth-important-ytl-power-231043491.html"><u class="StretchedBox"></u>If EPS Growth Is Important To You, YTL Power International Berhad (KLSE:YTLPOWR) Presents An Opportunity</a></h3><p class="Fz(14px) Lh(19px) Fz(13px)--sm1024 Lh(17px)--sm1024 LineClamp(2,38px) LineClamp(2,34px)--sm1024 M(0)">It's common for many investors, especially those who are inexperienced, to buy shares in companies with a good story...</p></div>


In [131]:
print("Source: ", div_tags[1].find('div').text)
print("Head Line : {}".format(div_tags[1].find('a').text))

Source:  Simply Wall St.
Head Line : If EPS Growth Is Important To You, YTL Power International Berhad (KLSE:YTLPOWR) Presents An Opportunity


In [132]:
print("Image URL: ",div_tags[1].findParent().find('img')['src'])

Image URL:  https://s.yimg.com/uu/api/res/1.2/WXDapnUsnhxaAn2W.9uY2g--~B/Zmk9c3RyaW07aD0xMjM7cT04MDt3PTIyMDthcHBpZD15dGFjaHlvbg--/https://media.zenfs.com/en/simply_wall_st__316/e44b48510541f08ce6d7c1a7b2f6b239.cf.jpg


In [133]:
#Download a webpage and return a beautiful soup doc
def get_page(url):
    response = requests.get(url)
    if not response.ok:
        print('Status code:', response.status_code)
        raise Exception('Failed to load page {}'.format(url))
    page_content = response.text
    doc = BeautifulSoup(page_content, 'html.parser')
    return doc

#Get the list of tags containing news information
def get_news_tags(doc):
    news_class = "Ov(h) Pend(44px) Pstart(25px)" ## class name of div tag 
    news_list  = doc.find_all('div', {'class': news_class})
    return news_list

#Get the news data point and return dictionary
def parse_news(news_tag):
    news_source = news_tag.find('div').text #source
    news_headline = news_tag.find('a').text #heading
    news_url = news_tag.find('a')['href'] #link
    news_content = news_tag.find('p').text #content
    news_image = news_tag.findParent().find('img')['src'] #thumb image
    return { 'source' : news_source,
            'headline' : news_headline,
            'url' : BASE_URL + news_url,
            'content' : news_content,
            'image' : news_image
           }

#Get the yahoo finance market news and write them to CSV file
def scrape_yahoo_news(url, path=None):
    if path is None:
        path = '../dse-mini-project-web-scrapping/ads_miniproject/static/stock-market-news.csv'
        
    print('Requesting html page')
    doc = get_page(url)

    print('Extracting news tags')
    news_list = get_news_tags(doc)

    print('Parsing news tags')
    news_data = [parse_news(news_tag) for news_tag in news_list]

    print('Save the data to a CSV')
    # Manually setting column names
    columns = ['source', 'headline', 'url', 'content', 'image']
    # Create DataFrame with specified column names
    news_df = pd.DataFrame(news_data, columns=columns)

    # Check if the file exists
    file_exists = os.path.exists(path)

    # If the file doesn't exist, write the header row
    if not file_exists:
        with open(path, 'w', newline='') as file:
            writer = csv.writer(file)
            writer.writerow(columns)

    # Append data to the CSV file without writing header
    news_df.to_csv(path, mode='a', header=False, index=False)

    print('Done')
    return news_df 

In [134]:
news_df_ytl = scrape_yahoo_news('https://finance.yahoo.com/quote/6742.KL') #YTL Power
news_df_nvdia = scrape_yahoo_news('https://finance.yahoo.com/quote/NVDA/') #NVIDIA


Requesting html page
Extracting news tags
Parsing news tags
Save the data to a CSV
Done
Requesting html page
Extracting news tags
Parsing news tags
Save the data to a CSV
Done


In [138]:
#display all the data
stock_market_df = pd.read_csv('../dse-mini-project-web-scrapping/ads_miniproject/static/stock-market-news.csv')

In [139]:
# get the total number of news
print('Total number of news:', len(stock_market_df))

Total number of news: 18


In [140]:
stock_market_df.head(3)

,source,headline,url,content,image
0,Simply Wall St.,YTL Power International Berhad Second Quarter ...,https://finance.yahoo.com/news/ytl-power-inter...,YTL Power International Berhad ( KLSE:YTLPOWR ...,https://s.yimg.com/uu/api/res/1.2/Un23csOlY9IT...
1,Simply Wall St.,"If EPS Growth Is Important To You, YTL Power I...",https://finance.yahoo.com/news/eps-growth-impo...,"It's common for many investors, especially tho...",https://s.yimg.com/uu/api/res/1.2/WXDapnUsnhxa...
2,The Wall Street Journal,"Tech, Media & Telecom Roundup: Market Talk",https://finance.yahoo.com/m/434e3156-3caa-38e5...,"Find insights on CAE, momentum for fintech, Eu...",https://s.yimg.com/uu/api/res/1.2/oeCZcEa3urOC...


In [144]:
#check duplicate news
duplicate_news = stock_market_df.duplicated(subset=['headline'])
duplicate_news.sum()

0

In [74]:


# Re-load the news data from CSV
news_df = pd.read_csv('../dse-mini-project-web-scrapping/ads_miniproject/static/stock-market-news.csv')

# Define a function to calculate sentiment polarity
def calculate_sentiment(text):
    try:
        return TextBlob(text).sentiment.polarity
    except:
        return None

# Apply the sentiment analysis on the 'content' column
news_df['sentiment_polarity'] = news_df['content'].apply(calculate_sentiment)

news_df.to_csv('../dse-mini-project-web-scrapping/ads_miniproject/static/stock-market-news.csv', index=False)

# Display the first few entries with sentiment polarity
news_df[['source', 'headline', 'sentiment_polarity']].head()


,source,headline,sentiment_polarity
0,Simply Wall St.,YTL Power International Berhad Second Quarter ...,0.00
1,Simply Wall St.,"If EPS Growth Is Important To You, YTL Power I...",0.16
2,The Wall Street Journal,"Tech, Media & Telecom Roundup: Market Talk",0.50
3,The Wall Street Journal,Energy & Utilities Roundup: Market Talk,0.50
4,Simply Wall St.,Is YTL Power International Berhad's (KLSE:YTLP...,0.80


# Sentiment Analysis Results for YTL Power International Berhad News

| No. | Headline | Sentiment Polarity | Sentiment Description |
|-----|----------|--------------------|-----------------------|
| 1. | YTL Power International Berhad Second Quarter Earnings | 0.00 | Neutral sentiment |
| 2. | If EPS Growth Is Important To You, YTL Power International Berhad Is Well Worth Watching | 0.16 | Slightly positive sentiment |
| 3. | Tech, Media & Telecom Roundup: Market Talk | 0.50 | Positive sentiment |
| 4. | Energy & Utilities Roundup: Market Talk | 0.50 | Positive sentiment |
| 5. | Is YTL Power International Berhad's (KLSE:YTLPWR) 4.3% Dividend Worth Your Time? | 0.80 | Very positive sentiment |

The sentiment scores indicate the tone of news articles related to YTL Power International Berhad. A score closer to 1 suggests a more positive sentiment, while a score closer to -1 suggests a more negative sentiment. Neutral sentiment is represented by a score of 0. These sentiment scores can provide insight into the public perception and media portrayal of the company, which could influence investor behavior and potentially correlate with stock market performance.

In [76]:
# get the average sentiment polarity
average_sentiment = news_df['sentiment_polarity'].mean()
average_sentiment

0.23063356566346999

The average sentiment polarity score of 0.2306 suggests that the overall sentiment of news articles related to YTL Power International Berhad is slightly positive. This indicates that, on average, the news content leans more towards positive than negative, which could be reflective of generally favorable news or optimistic outlooks in the reporting on the company. This sentiment may influence investor sentiment and could correlate with the company's stock performance.